In [2]:
# 단면적 계산
def cal_area(center_a,center_b,d,theta_1,theta_2,R):

    try :
        theta = theta_1 - theta_2

        point_1 = center_a +(center_b*center_a)/(R*tf.sin(theta_1))

        point_2 = center_a +(center_b*(center_a-d))/(R*tf.sin(theta_2))

        fan_shape = (1/2) * theta * (pow(R,2))

        triangle_1 = (1/2)  * (0-point_1) * (R*tf.sin(theta_1)+center_b)

        triangle_2 = (1/2)  * (d - point_2) * (R*tf.sin(theta_2)+center_b)

        triangle_3 = (1/2)  * (point_2 - point_1) * (0-center_b)

        area = fan_shape - triangle_1 + triangle_2 - triangle_3

    except :
        area = 0

    # print(f'theta_1: {math.degrees(theta_1)} theta_2: {math.degrees(theta_2)} theta: {math.degrees(theta)}')

    return area

In [3]:
# Import TensorFlow and NumPy
# 초기 변수들 지정
import tensorflow as tf
import numpy as np
import scipy as sp

# Set data type
DTYPE='float64'
tf.keras.backend.set_floatx(DTYPE)

# Define contact line velocity
# theta_0 : equilibrium contact angle
# theta_d : dynamic contact angle
def fun_contact_line_velocity( f_c,i_t,theta_0, theta_d ):
    one = tf.constant(1, dtype=tf.float64)
    return -f_c*i_t*(-tf.cos(theta_0)+tf.cos(theta_d))
# Set constants
pi = tf.constant(np.pi, dtype=tf.float64)
p_f_c = tf.constant(-2.02, dtype=tf.float64) # positive_friction_coefficient
n_f_c = tf.constant(-5.64, dtype=tf.float64) # nagative_friction_coefficient
i_t = tf.constant(7.6408/1000, dtype=tf.float64) #interfacial tension between the two liquids
d = tf.constant(0.321/1000, dtype=tf.float64) #gab

thetal_0 = tf.constant(np.deg2rad(94), dtype=tf.float64)
thetal_d = tf.constant(np.deg2rad(55), dtype=tf.float64)
thetar_0 = tf.constant(np.deg2rad(59),dtype=tf.float64)
thetar_d = tf.constant(np.deg2rad(109),dtype=tf.float64)

# 반지름
r_0 = tf.constant(np.deg2rad(109), dtype=tf.float64)
# 원의 중심 x좌표
cneter_a = tf.constant(-0.0000566758, dtype=tf.float64)
# 원의 중심 y좌표
cneter_b = tf.constant(-0.0006874250504951296, dtype=tf.float64)

#  세타1 좌표
theta1_0 = tf.constant(np.pi-np.deg2rad(94),dtype=tf.float64)
#  세타2 좌표
theta2_0 =  thetar_0

vl = tf.constant(-fun_contact_line_velocity(n_f_c ,i_t,thetal_0, thetal_d ),shape=(1,1) , dtype=tf.float64)
vr = tf.constant(-fun_contact_line_velocity(p_f_c ,i_t,thetar_0 , thetar_d ),shape=(1,1) , dtype=tf.float64)

# initial_data = [a0 ,b0,r0 ,theta_l0,theta_r0 ]
# cal_area_theta(center_a,center_b,d,theta_l,theta_r,R):
initial_area = tf.constant(5.375716695978319e-08,shape=(1,1) , dtype=tf.float64)
print(initial_area)
#thetar_d = tf.constant(initial_area, dtype=tf.float64)

# Define residual of the condition
def fun_r(yl_t , yr_t ,vl,vr,area_t ):
    return ((yl_t-vl) + (yr_t-vr)+area_t)

tf.Tensor([[5.3757167e-08]], shape=(1, 1), dtype=float64)


In [4]:
# 트레이닝 데이터(t)
# Set random seed for reproducible results
tf.random.set_seed(0)

# 시간(0~0.0280초까지)
t_data = tf.random.uniform(shape=[1,10000],minval=0.0,maxval=0.0280, dtype=DTYPE)
t_data = tf.reshape(t_data,shape=(-1,1))

t_data = tf.constant(t_data,dtype=DTYPE)
# 정렬( 시계열 데이터 이기 때문)
t_data = tf.sort(t_data,axis=0)
print(t_data)

tf.Tensor(
[[1.27272500e-06]
 [8.12988319e-06]
 [1.36279060e-05]
 ...
 [2.79928773e-02]
 [2.79933925e-02]
 [2.79937181e-02]], shape=(10000, 1), dtype=float64)


In [5]:
# 모델 구축
def init_model(num_hidden_layers=8, num_neurons_per_layer=20):
    # Initialize a feedforward neural network
    model = tf.keras.Sequential()

    # Input is one-dimensional (time)
    model.add(tf.keras.Input(1,))

    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons_per_layer,
            activation=tf.keras.activations.get('tanh'),
            input_dim=t_data.shape[1],
            kernel_initializer='glorot_normal'))

    # Output is one-dimensional (center_a , center_b , R , theat_l , thetah_r)
    model.add(tf.keras.layers.Dense(5,))

    return model

In [6]:
# get residual
def get_r(model, t_data):

    # A tf.GradientTape is used to compute derivatives in TensorFlow
    with tf.GradientTape(persistent=True) as tape:
        # Split t to compute partial derivatives
        t = t_data
        # Variables t are watched during tape
        tape.watch(t)

        # 아웃풋을 a,b,R,theta_1,theta_2로 각각 분리
        u = model(t_data)
        a = tf.reshape(u[:,0],shape=(-1,1))
        b = tf.reshape(u[:,1],shape=(-1,1))
        R = tf.reshape(u[:,2],shape=(-1,1))
        theta_1 =tf.reshape(u[:,3],shape=(-1,1))
        theta_2 = tf.reshape(u[:,4],shape=(-1,1))

        # get area
        area = cal_area(a,b,d,theta_1,theta_2,R)

    # get gradient
    a_t = tape.gradient(a,t)
    b_t = tape.gradient(b,t)
    R_t = tape.gradient(R,t)
    theta_1_t = tape.gradient(theta_1,t)
    theta_2_t = tape.gradient(theta_2,t)
    area_t = tape.gradient(area, t)


    yl_t = R_t*tf.sin(theta_1)+R*tf.cos(theta_1)*theta_1_t +b_t
    yr_t = R_t*tf.sin(theta_2)+R*tf.cos(theta_2)*theta_2_t +b_t


    del tape

    return fun_r(yl_t , yr_t ,vl ,vr,area_t)

In [7]:
def compute_loss(model,t_data):


    # Compute phi^r
    r = get_r(model, t_data)
    phi_r = tf.reduce_mean(tf.square(r))

    # Initialize loss - version1
    t_0 = tf.zeros((1,1))

    # Add phi^0 and phi^b to the loss
    # u = model(t_0)
    # pred_a = tf.reshape(u[:,0],shape=(-1,1))
    # pred_b = tf.reshape(u[:,1],shape=(-1,1))
    # pred_R = tf.reshape(u[:,2],shape=(-1,1))
    # pred_theta_1 =tf.reshape(u[:,3],shape=(-1,1))
    # pred_theta_2 = tf.reshape(u[:,4],shape=(-1,1))
    # pred_pred_area_0 = cal_area(pred_a,pred_b,d,pred_theta_1,pred_theta_2,pred_R)
    # loss =phi_r+tf.reduce_mean(tf.square(pred_pred_area_0 -initial_area))

    # version2 -> 학습 초기에는 t=0일때 [0,0,0,0,0] 아웃풋이 나오는데 이의 단면적을 계산하면 nan이 나옴


    # Initialize loss - version2
    #loss = phi_r

    # Add phi^0 and phi^b to the loss
    u = model(t_data)
    pred_a = tf.reshape(u[:,0],shape=(-1,1))
    pred_b = tf.reshape(u[:,1],shape=(-1,1))
    pred_R = tf.reshape(u[:,2],shape=(-1,1))
    pred_theta_1 =tf.reshape(u[:,3],shape=(-1,1))
    pred_theta_2 = tf.reshape(u[:,4],shape=(-1,1))
    pred_pred_area_0 = cal_area(pred_a,pred_b,d,pred_theta_1,pred_theta_2,pred_R)
    loss =phi_r+tf.reduce_mean(tf.square(pred_pred_area_0 -initial_area))


    return loss


In [8]:
def get_grad(model,t_data):

    with tf.GradientTape(persistent=True) as tape:
        # This tape is for derivatives with
        # respect to trainable variables
        tape.watch(model.trainable_variables)
        loss = compute_loss(model,t_data)

    g = tape.gradient(loss, model.trainable_variables)
    del tape

    return loss, g

In [9]:
# Initialize model aka u_\theta
model = init_model()

# We choose a piecewise decay of the learning rate, i.e., the
# step size in the gradient descent type algorithm
# the first 1000 steps use a learning rate of 0.01
# from 1000 - 3000: learning rate = 0.001
# from 3000 onwards: learning rate = 0.0005

lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,3000],[1e-2,1e-3,5e-4])

# Choose the optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr)

In [10]:
from time import time

# Define one training step as a TensorFlow function to increase speed of training
@tf.function
def train_step():
    # Compute current loss and gradient w.r.t. parameters
    loss, grad_theta = get_grad(model,t_data)

    # Perform gradient descent step
    optim.apply_gradients(zip(grad_theta, model.trainable_variables))

    return loss

# Number of training epochs
N = 5000
hist = []

# Start timer
t0 = time()

for i in range(N+1):

    loss = train_step()

    # Append current loss to hist
    hist.append(loss.numpy())

    # Output current loss after 50 iterates
    if i%50 == 0:
        print('It {:05d}: loss = {:10.8e}'.format(i,loss))

# Print computation time
print('\nComputation time: {} seconds'.format(time()-t0))

It 00000: loss = 1.17260698e-02
It 00050: loss = 8.63090997e-05
It 00100: loss = 1.25146999e-06
It 00150: loss = 7.58080489e-07
It 00200: loss = 7.38395991e-07
It 00250: loss = 7.28109235e-07
It 00300: loss = 7.16845646e-07
It 00350: loss = 7.04675102e-07
It 00400: loss = 6.91701660e-07
It 00450: loss = 6.78016550e-07
It 00500: loss = 6.63701581e-07
It 00550: loss = 6.48831432e-07
It 00600: loss = 6.33475142e-07
It 00650: loss = 6.17697119e-07
It 00700: loss = 6.01557832e-07
It 00750: loss = 5.85114312e-07
It 00800: loss = 5.68420505e-07
It 00850: loss = 5.51527545e-07
It 00900: loss = 5.34483952e-07
It 00950: loss = 5.17335790e-07
It 01000: loss = 5.00126787e-07
It 01050: loss = 4.98069193e-07
It 01100: loss = 4.96261579e-07
It 01150: loss = 4.94392670e-07
It 01200: loss = 4.92462392e-07
It 01250: loss = 4.90470648e-07
It 01300: loss = 4.88417313e-07
It 01350: loss = 4.86302238e-07
It 01400: loss = 4.84125257e-07
It 01450: loss = 4.81886188e-07
It 01500: loss = 4.79584838e-07
It 01550

In [11]:
test_data=tf.linspace(0.0, 0.0280, 281)
test_data = tf.reshape(test_data,shape=(-1,1))
model(test_data)

<tf.Tensor: shape=(281, 5), dtype=float64, numpy=
array([[ 0.62932965, -0.3389331 , -0.57900239,  0.52310261,  0.29269733],
       [ 0.62934327, -0.33892827, -0.57901045,  0.52309935,  0.29270861],
       [ 0.6293569 , -0.33892344, -0.57901852,  0.52309609,  0.29271989],
       ...,
       [ 0.63314754, -0.33756819, -0.5812427 ,  0.52216773,  0.29589035],
       [ 0.63316138, -0.3375632 , -0.58125075,  0.52216426,  0.29590205],
       [ 0.63317521, -0.33755821, -0.5812588 ,  0.52216079,  0.29591374]])>

In [ ]:
u=model(tf.zeros((1,1)))
print(u)